In [ ]:
%matplotlib inline

### Parse and store SNOMED taxonomy

In [35]:
from rdflib.extras.external_graph_libs import *
from rdflib import Graph
import networkx as nx
import pathlib

In [68]:
def node_to_id(node):
    return node.split('/')[-1]

In [104]:
path = pathlib.Path("./data/snomed_taxonomy.rdf")

# every node with 'http://snomed.info/id/SNOMED_ID' is child of 'http://www.w3.org/2002/07/owl#Class'
# 'http://snomed.info/id/138875005' is top node --> SNOMED CT Concept (SNOMED RT+CTV3)
# graph is directed in top direction (that is, a neighbor is/are only the direct parent(s))

graph = Graph()
graph.parse(path, format="application/rdf+xml")
nx_graph = rdflib_to_networkx_multidigraph(graph)

id_node_mapping = {}
for n in list(nx_graph.nodes()):
    _id = node_to_id(n)
    if (not _id.isnumeric()) or (node_to_id(n) == "138875005"):
        nx_graph.remove_node(n)  # Remove meta nodes (e.g. 'owl#Class') and top level node 'SNOMED CT Concept (SNOMED RT+CTV3)'
        continue
    id_node_mapping[_id] = n

In [67]:
def id_to_node(snomed_id: str, mapping_dict: dict = id_node_mapping):
    node = mapping_dict.get(snomed_id, None)
    return node

In [96]:
def parents_of_id(snomed_id: str, mapping_dict: dict = id_node_mapping):
    for neighbor in nx_graph.neighbors(id_to_node(snomed_id, mapping_dict)):
        _id = node_to_id(neighbor)
        if not _id.isnumeric():
            continue
        yield _id

In [117]:
def children_of_id(snomed_id: str, mapping_dict: dict = id_node_mapping):
    for predecessor in nx_graph.predecessors(id_to_node(snomed_id, mapping_dict)):
        _id = node_to_id(predecessor)
        if not _id.isnumeric():
            continue
        yield _id

---
### Read SNOMED Interface Terminology

In [73]:
import collections

In [78]:
interface_terminology_path = pathlib.Path("./data/SCT-GIT_de_drugs_nlp.dat")
interface_terminology = collections.defaultdict(list)

for line in interface_terminology_path.open('r', encoding='utf-8').readlines():
    # first name in the resulting list for each entry is the preferred name
    snomed_id, internal_id, concept_name, interface_name = line.strip().split("\t")
    interface_terminology[snomed_id].append(interface_name)
top_level_concepts = [node_to_id(n) for n in nx_graph.nodes() if
                      nx_graph.out_degree(n) == 0 and node_to_id(n) in interface_terminology]

In [81]:
len(interface_terminology)

313548

In [82]:
len(id_node_mapping)

366790

In [109]:
len(top_level_concepts)

19

---
### Create/upload concepts to TOP FW

In [123]:
import requests
import uuid

In [ ]:
# Entity Model to be uploaded to TOP FW
{
    "entityType": "single_concept",
    "id": "SOME_UUID",
    "titles": [
        {
            "language": "de",
            "text": "INTERFACE_NAME_PREFERRED"
        }
    ],
    "synonyms": [
        {
            "language": "de",
            "text": "INTERFACE_NAME_next"
        },
        ...
    ],
    "codes": [
        {
            "codeSystem": {
                "uri": "http://snomed.info/id"
            },
            "code": "SNOMED_ID",
        }
    ],
    "superConcepts": [
        
    ]
}